<a href="https://colab.research.google.com/github/Gaabiin5/Hugging_Face_Formation/blob/main/multiagent_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solving a complex task with a multi-agent hierarchy

This notebook is part of the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course), a free Course from beginner to expert, where you learn to build Agents.

![Agents course share](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/communication/share.png)

The reception is approaching! With your help, Alfred is now nearly finished with the preparations.

But now there's a problem: the Batmobile has disappeared. Alfred needs to find a replacement, and find it quickly.

Fortunately, a few biopics have been done on Bruce Wayne's life, so maybe Alfred could get a car left behind on one of the movie set, and re-engineer it up to modern standards, which certainly would include a full self-driving option.

But this could be anywhere in the filming locations around the world - which could be numerous.

So Alfred wants your help. Could you build an agent able to solve this task?

> 👉 Find all Batman filming locations in the world, calculate the time to transfer via a cargo plane to there, and represent them on a map, with a color varying by a cargo plane transfer time. Also represent some supercar factories with the same cargo plane transfer time.

Let's build this!

# Warning

This notebook does not functions well because of the switch to local models. Many errors who cannot be fixed because we have not build the entire agent


In [ ]:
%pip install "litellm>=1.27.6"

In [1]:
import litellm

if not hasattr(litellm, "custom_provider_map"):
    litellm.custom_provider_map = {}

In [ ]:
%pip install 'smolagents[litellm]' plotly geopandas shapely kaleido -q

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
# We first make a tool to get the cargo plane transfer time.
import math
from typing import Optional, Tuple

from smolagents import tool


@tool
def calculate_cargo_travel_time(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
    cruising_speed_kmh: Optional[float] = 750.0,  # Average speed for cargo planes
) -> float:
    """
    Calculate the travel time for a cargo plane between two points on Earth using great-circle distance.

    Args:
        origin_coords: Tuple of (latitude, longitude) for the starting point
        destination_coords: Tuple of (latitude, longitude) for the destination
        cruising_speed_kmh: Optional cruising speed in km/h (defaults to 750 km/h for typical cargo planes)

    Returns:
        float: The estimated travel time in hours

    Example:
        >>> # Chicago (41.8781° N, 87.6298° W) to Sydney (33.8688° S, 151.2093° E)
        >>> result = calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093))
    """

    def to_radians(degrees: float) -> float:
        return degrees * (math.pi / 180)

    # Extract coordinates
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    # Earth's radius in kilometers
    EARTH_RADIUS_KM = 6371.0

    # Calculate great-circle distance using the haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.asin(math.sqrt(a))
    distance = EARTH_RADIUS_KM * c

    # Add 10% to account for non-direct routes and air traffic controls
    actual_distance = distance * 1.1

    # Calculate flight time
    # Add 1 hour for takeoff and landing procedures
    flight_time = (actual_distance / cruising_speed_kmh) + 1.0

    # Format the results
    return round(flight_time, 2)


print(calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093)))

22.82


For the model provider, we use Together AI, one of the new [inference providers on the Hub](https://huggingface.co/blog/inference-providers)!

Regarding the GoogleSearchTool: this requires either having setup env variable `SERPAPI_API_KEY` and passing `provider="serpapi"` or having `SERPER_API_KEY` and passing `provider=serper`.

If you don't have any Serp API provider setup, you can use `DuckDuckGoSearchTool` but beware that it has a rate limit.

In [3]:
import os 
from dotenv import load_dotenv

load_dotenv()

serpapi_api_key = os.getenv("SERPAPI_API_KEY")


#os.environ["SERPAPI_API_KEY"] = userdata.get('SERPAPI_API_KEY')

In [13]:
from smolagents import ToolCallingAgent, DuckDuckGoSearchTool, InferenceClientModel, LiteLLMModel

model = LiteLLMModel(
        # model_id="ollama_chat/deepseek-coder:6.7b-instruct",  # Or try other Ollama-supported models
        # model_id="ollama_chat/phi4-mini",
        model_id="ollama_chat/openchat",
        api_base="http://127.0.0.1:11434",  # Default Ollama local server
        num_ctx=8192,
        # temperature=0.2,  # rLess creativity
        # max_tokens=512,   # Limit the lenght of the response
        system_message="""You are a Python reasoning agent. Your job is to solve user instructions by generating clean, robust Python code using the available tools.

Constraints:
- All external tool responses must be validated before being used.
- Never assume that a string is well-formed or that a function response is in the expected format.
- Always check the length and content of strings before doing `.split()` or type conversions like `float()`.
- Use `try/except` or `if` conditions to avoid runtime errors when handling data from tools or user inputs.
- If working with location coordinates, always confirm they are valid and contain two numeric parts (latitude and longitude).
- Do not include markdown or text explanations — return only valid Python code.

Format:
1. Parse and validate all inputs safely.
2. Only use available tools and assume they may return errors or unexpected formats.
3. Print relevant output for each major step.

Begin solving the following task:

{{user_input}}

"""
)

In [5]:
import os
from PIL import Image
from smolagents import CodeAgent, GoogleSearchTool, InferenceClientModel, VisitWebpageTool


# model = InferenceClientModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")

We can start with creating a baseline, simple agent to give us a simple report.

In [6]:
task = """Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.
Also give me some supercar factories with the same cargo plane transfer time."""

In [ ]:
# from google.colab import userdata
# import os

# os.environ["SERPAPI_API_KEY"] = userdata.get('SERPAPI_API_KEY')

ModuleNotFoundError: No module named 'google.colab'

In [14]:
agent = CodeAgent(
    model=model,
    tools=[GoogleSearchTool(), VisitWebpageTool(), calculate_cargo_travel_time],
    additional_authorized_imports=["pandas"],
    max_steps=20,
)

In [15]:
result = agent.run(task)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/openchat ───────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # List of Batman filming locations in the world                                                                  
  batman_locations = web_search(query="Batman filming locations")                                                  
  print("Batman filming locations:\n", batman_locations)                                                           
                                                                                                                   
  # Calculate the travel time from Gotham's coordinates to each location using a cargo plane                       
  gotham_coordinates = (40.7128, -74.0060)                                                                         
  for location in batman_locations:                                                                                
      location_coordinates = float(location.split("latitude")[-1]), float(location.split("longitude")[-1])         
      travel_time = calculate_cargo_travel_time(gotham_coordinates, location_coordinates)                          
      print(f"Travel time to {location} from Gotham: {travel_time} hours")                                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Batman filming locations:
 ## Search Results
0. [The Batman (2022) - Filming & production](https://www.imdb.com/title/tt1877830/locations/)
Source: IMDb

Filming locations: Necropolis Cemetery, Glasgow, Scotland, UK (Batman and Selina leaving the cemetery) Helpful•87 1
St. George's Hall, Liverpool, England, UK

1. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022-2.php)
Source: The Worldwide Guide To Movie Locations

The Batman filming location: St George's Hall, Liverpool · The Batman filming location: Royal Liver Building, Pier 
Head, Liverpool · The Batman filming location: ...

2. [Batman (1989) - Filming & production](https://www.imdb.com/title/tt0096895/locations/)
Source: IMDb

Filming locations ; Knebworth House, Knebworth, Hertfordshire, England, UK. (Wayne Manor; exterior) · 21 ; Acton 
Lane Power Station, Acton Lane, Acton, London, ...

3. [Where was The Batman filmed? ALL the Filming Locations 
...](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
Source: Atlas of Wonders

The Batman was primarily filmed in the United Kingdom. Most of the recognizable buildings of this new version of 
Gotham City are located in Liverpool.

4. [12 Batman Movie Locations You Can 
Visit!](https://www.travelandleisureasia.com/global/destinations/batman-movie-locations-you-can-visit/)
Date published: Jan 10, 2023
Source: Travel and Leisure Asia

Most of the filming of Batman movies is done in the Warner Bros studios and across the US, including New York and 
Pittsburgh. You will notice ...

5. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022.php)
Source: The Worldwide Guide To Movie Locations

Film locations for The Batman (2022) in Liverpool, London, Glasgow ... The Batman filming location: James R 
Thompson Center, West Randolph Street, Chicago.

6. [Batman Movies and Filming Locations in Los 
Angeles](https://www.traveltodayla.com/post/batman-movies-in-la?srsltid=AfmBOopUSIwXEkM1A99hTQCPdr47_onIRwVl_zVi3B9
pZYLdAEH9EwFn)
Date published: Apr 20, 2023
Source: LA Today

Batman Movies and Filming Locations in LA · Cliffs of Marineland · Kirkeby Center · Gateway East Building · Wayne 
Manor · Bronson Caves · Batman ...

7. [In Time / Batman location? : 
r/whereisthis](https://www.reddit.com/r/whereisthis/comments/146xczi/in_time_batman_location/)
Source: Reddit · r/whereisthis

Dark Knight is filming in LA :: The Dark Knight Rises is filming at 433 S Spring St in L.A. (9/16/2011 – 09/17/2011
4:00 AM – 10:00 PM).

Code execution failed at line 'for location in batman_locations:
    location_coordinates = float(location.split("latitude")[-1\]), float(location.split("longitude")[-1\])
    travel_time = calculate_cargo_travel_time(gotham_coordinates, location_coordinates)
    print(f"Travel time to {location} from Gotham: {travel_time} hours")' due to: ValueError: could not convert 
string to float: '#'

...

Code execution failed at line 'for location in batman_locations:
    location = filter_location(location)
    if not location:
        continue
    location_coordinates = float(location.split("latitude")[-1\]), float(location.split("longitude")[-1\])
    travel_time = calculate_cargo_travel_time(gotham_coordinates, location_coordinates)
    print(f"Travel time to {location} from Gotham: {travel_time} hours")' due to: ValueError: could not convert 
string to float: ''

[Step 4: Duration 19.44 seconds| Input tokens: 20,966 | Output tokens: 1,215]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

[Step 5: Duration 4.79 seconds]

KeyboardInterrupt: 

In [9]:
result

"## Search Results\n0. [supercar (Sorted by Popularity Ascending)](https://www.imdb.com/search/title/?keywords=supercar)\nSource: IMDb\n\nDavid Hasselhoff in Knight Rider (1982). 1. Knight Rider · It Takes Two (1988). 2. It Takes Two · Cars Mater-National (2007). 3. Cars Mater-National.\n\n1. [Supercar Films](https://www.youtube.com/playlist?list=PLbFzcFIntSnnAKy63UjenVZqx-Z85GaRO)\nSource: YouTube · Car Films\n\nSupercar Films · Audi R8 V8 supercar review / story · Ferrari 308 GTB Classic Car Review · The Story of the AC 3000 - Rare Classic Car · Porsche 928 classic car ...\n\n2. [Best Car Movies Ever (for True Car Fanatics)](https://www.supercars.net/blog/12-best-car-movies-ever-true-car-fanatics/)\nSource: Supercars.net\n\n12 Best Car Movies Ever · 1. Senna · 2. The Italian Job · 3. C'était un Rendez-vous · 4. Mad Max 2: The Road Warrior · 5. Bullitt · 6. Le Mans · 7. Ronin · 8.\n\n3. [The 39 Greatest Car Movies Ever Made](https://www.vulture.com/2015/04/29-greatest-car-movies-ever.

### Observations 

[92m14:52:10 - LiteLLM:ERROR[0m: utils.py:799 - litellm.utils.py::function_setup() - [Non-Blocking] Error in function_setup
Traceback (most recent call last):
  File "/home/gan/venv/lib/python3.12/site-packages/litellm/utils.py", line 526, in function_setup
    custom_llm_setup()
  File "/home/gan/venv/lib/python3.12/site-packages/litellm/utils.py", line 357, in custom_llm_setup
    for custom_llm in litellm.custom_provider_map:
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: module 'litellm' has no attribute 'custom_provider_map'

### Solutions 

- Utiliser un InferenceClient comme prevu de base 
- Completer le LiteLLM

We could already improve this a bit by throwing in some dedicated planning steps, and adding more prompting.

In [16]:
agent.planning_interval = 4

detailed_report = agent.run(f"""
You're an expert analyst. You make comprehensive reports after visiting many websites.
Don't hesitate to search for many queries at once in a for loop.
For each data point that you find, visit the source url to confirm numbers.

{task}
""")

print(detailed_report)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You're an expert analyst. You make comprehensive reports after visiting many websites.                          │
│ Don't hesitate to search for many queries at once in a for loop.                                                │
│ For each data point that you find, visit the source url to confirm numbers.                                     │
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/openchat ───────────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
 ## Part 1: Facts Survey
### 1.1. Facts given in the task
* We are in Gotham (coordinates: 40.7128° N, 74.0060° W)

### 1.2. Facts to look up
* Batman filming locations worldwide
* Coordinates of these locations
* Time to transfer via cargo plane from Gotham to each location
* Supercar factories worldwide
* Coordinates of these factories
* Time to transfer via cargo plane from Gotham to each factory

### 1.3. Facts to derive
* Batman filming locations in a pandas dataframe with their respective transfer times
* List of supercar factories in a pandas dataframe with their respective transfer times

## Part 2: Plan
1. Web scrape or use an API to collect all known Batman filming locations and their coordinates worldwide.
2. For each filming location, calculate the time to transfer via cargo plane from Gotham to that location using the
`calculate_cargo_travel_time` function.
3. Create a pandas dataframe with two columns: 'Location' and 'Transfer Time'. Fill these with the respective 
filming locations and their calculated transfer times.
4. Web scrape or use an API to collect all known supercar factories and their coordinates worldwide.
5. For each supercar factory, calculate the time to transfer via cargo plane from Gotham to that factory using the 
`calculate_cargo_travel_time` function.
6. Create a second pandas dataframe with two columns: 'Factory' and 'Transfer Time'. Fill these with the respective
supercar factories and their calculated transfer times.
7. Combine both dataframes into one and sort them by increasing order of Transfer Time. This will be our final 
result.
8. Use the `final_answer` function to return the combined dataframe as the final answer to the problem.

\n
```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
  from datetime import timedelta                                                                                   
                                                                                                                   
  # Step 1: Collecting Batman filming locations and their coordinates                                              
  batman_locations = web_search(query="Batman filming locations worldwide")                                        
  print(f"Batman filming locations found: {len(batman_locations)}")                                                
                                                                                                                   
  # Step 2: Calculating transfer times to each Batman location                                                     
  batman_df = pd.DataFrame({'Location': batman_locations, 'Transfer Time': []})                                    
  for location in batman_locations:                                                                                
      try:                                                                                                         
          time_to_location = calculate_cargo_travel_time(Gotham=40.7128, 74.0060, location[1], location[0])        
          batman_df['Transfer Time'].append(time_to_location)                                                      
      except:                                                                                                      
          print("Failed to calculate transfer time for location:", location)                                       
                                                                                                                   
  # Step 3: Creating a dataframe with Batman filming locations and their transfer times                            
  batman_df = batman_df.fillna(0).round(2)                                                                         
  print(f"Batman filming locations and transfer times:\n{batman_df}")                                              
                                                                                                                   
  # Step 4: Collecting supercar factory locations and their coordinates                                            
  supercar_factories = web_search(query="Supercar factories worldwide")                                            
  print(f"Supercar factories found: {len(supercar_factories)}")                                                    
                                                                                                                   
  # Step 5: Calculating transfer times to each Supercar factory                                                    
  supercar_df = pd.DataFrame({'Factory': supercar_factories, 'Transfer Time': []})                                 
  for factory in supercar_factories:                                                                               
      try:                                                                                                         
          time_to_factory = calculate_cargo_travel_time(Gotham=40.7128, 74.0060, factory[1], factory[0])           
          supercar_df['Transfer Time'].append(time_to_factory)                                                     
      except:                                                                                                      
          print("Failed to calculate transfer time for factory:", factory)                                         
                                                                                                                   
  # Step 6: Creating a dataframe with Supercar factories 

Execution logs:
Batman filming locations:
 ## Search Results
0. [The Batman (2022) - Filming & production](https://www.imdb.com/title/tt1877830/locations/)
Source: IMDb

Filming locations: Necropolis Cemetery, Glasgow, Scotland, UK (Batman and Selina leaving the cemetery) Helpful•87 1
St. George's Hall, Liverpool, England, UK

1. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022-2.php)
Source: The Worldwide Guide To Movie Locations

The Batman filming location: St George's Hall, Liverpool · The Batman filming location: Royal Liver Building, Pier 
Head, Liverpool · The Batman filming location: ...

2. [Batman (1989) - Filming & production](https://www.imdb.com/title/tt0096895/locations/)
Source: IMDb

Filming locations ; Knebworth House, Knebworth, Hertfordshire, England, UK. (Wayne Manor; exterior) · 21 ; Acton 
Lane Power Station, Acton Lane, Acton, London, ...

3. [Where was The Batman filmed? ALL the Filming Locations 
...](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
Source: Atlas of Wonders

The Batman was primarily filmed in the United Kingdom. Most of the recognizable buildings of this new version of 
Gotham City are located in Liverpool.

4. [12 Batman Movie Locations You Can 
Visit!](https://www.travelandleisureasia.com/global/destinations/batman-movie-locations-you-can-visit/)
Date published: Jan 10, 2023
Source: Travel and Leisure Asia

Most of the filming of Batman movies is done in the Warner Bros studios and across the US, including New York and 
Pittsburgh. You will notice ...

5. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022.php)
Source: The Worldwide Guide To Movie Locations

Film locations for The Batman (2022) in Liverpool, London, Glasgow ... The Batman filming location: James R 
Thompson Center, West Randolph Street, Chicago.

6. [Batman Movies and Filming Locations in Los 
Angeles](https://www.traveltodayla.com/post/batman-movies-in-la?srsltid=AfmBOopUSIwXEkM1A99hTQCPdr47_onIRwVl_zVi3B9
pZYLdAEH9EwFn)
Date published: Apr 20, 2023
Source: LA Today

Batman Movies and Filming Locations in LA · Cliffs of Marineland · Kirkeby Center · Gateway East Building · Wayne 
Manor · Bronson Caves · Batman ...

7. [In Time / Batman location? : 
r/whereisthis](https://www.reddit.com/r/whereisthis/comments/146xczi/in_time_batman_location/)
Source: Reddit · r/whereisthis

Dark Knight is filming in LA :: The Dark Knight Rises is filming at 433 S Spring St in L.A. (9/16/2011 – 09/17/2011
4:00 AM – 10:00 PM).

...

Code execution failed at line 'for location in batman_locations:
    coordinates_batman.append((float(location[1\]), float(location[0\])))' due to: InterpreterError: Could not 
index # with '1': IndexError: string index out of range

[Step 3: Duration 43.94 seconds| Input tokens: 17,175 | Output tokens: 1,931]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

[Step 4: Duration 8.05 seconds]

KeyboardInterrupt: 

In [ ]:
detailed_report

,Location,Travel Time (hours)
0,"Bridge of Sighs, Glasgow Necropolis, Glasgow, ...",8.6
1,"Wishart Street, Glasgow, Scotland, UK",8.6


Thanks to these quick changes, we obtained a much more concise report by simply providing our agent a detailed prompt, and giving it planning capabilities!

💸 But as you can see, the context window is quickly filling up. So **if we ask our agent to combine the results of detailed search with another, it will be slower and quickly ramp up tokens and costs**.

➡️ We need to improve the structure of our system.

## ✌️ Splitting the task between two agents

Multi-agent structures allow to separate memories between different sub-tasks, with two great benefits:
- Each agent is more focused on its core task, thus more performant
- Separating memories reduces the count of input tokens at each step, thus reducing latency and cost.

Let's create a team with a dedicated web search agent, managed by another agent.

The manager agent should have plotting capabilities to redact its final report: so let us give it access to additional imports, including `plotly`, and `geopandas` + `shapely` for spatial plotting.

In [ ]:
model = InferenceClientModel(
    "Qwen/Qwen2.5-Coder-32B-Instruct", provider="together", max_tokens=8096
)

web_agent = CodeAgent(
    model=model,
    tools=[
        GoogleSearchTool(provider="serper"),
        VisitWebpageTool(),
        calculate_cargo_travel_time,
    ],
    name="web_agent",
    description="Browses the web to find information",
    verbosity_level=0,
    max_steps=10,
)

The manager agent will need to do some mental heavy lifting.

So we give it the stronger model [DeepSeek-R1](https://huggingface.co/deepseek-ai/DeepSeek-R1), and add a `planning_interval` to the mix.

In [ ]:
# from google.colab import userdata
# import os
# os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
from smolagents.utils import encode_image_base64, make_image_url
from smolagents import OpenAIServerModel


def check_reasoning_and_plot(final_answer, agent_memory):
    multimodal_model = OpenAIServerModel("gpt-4o", max_tokens=8096)
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True


manager_agent = CodeAgent(
    model=InferenceClientModel("deepseek-ai/DeepSeek-R1", provider="together", max_tokens=8096),
    tools=[calculate_cargo_travel_time],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "shapely",
        "json",
        "pandas",
        "numpy",
    ],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)

Let us inspect what this team looks like:

In [ ]:
manager_agent.visualize()

CodeAgent | deepseek-ai/DeepSeek-R1
├── ✅ Authorized imports: ['geopandas', 'plotly', 'shapely', 'json', 'pandas', 'numpy']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name                        ┃ Description                           ┃ Arguments                             ┃
│   ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ calculate_cargo_travel_time │ Calculate the travel time for a cargo │ origin_coords (`array`): Tuple of     │
│   │                             │ plane between two points on Earth     │ (latitude, longitude) for the         │
│   │                             │ using great-circle distance.          │ starting point                        │
│   │                             │                                       │ destination_coords (`array`): Tuple   │
│   │                             │                                       │ of (latitude, longitude) for the      │
│   │                             │                                       │ destination                           │
│   │                             │                                       │ cruising_speed_kmh (`number`):        │
│   │                             │                                       │ Optional cruising speed in km/h       │
│   │                             │                                       │ (defaults to 750 km/h for typical     │
│   │                             │                                       │ cargo planes)                         │
│   │ final_answer                │ Provides a final answer to the given  │ answer (`any`): The final answer to   │
│   │                             │ problem.                              │ the problem                           │
│   └─────────────────────────────┴───────────────────────────────────────┴───────────────────────────────────────┘
└── 🤖 Managed agents:
    └── web_agent | CodeAgent | Qwen/Qwen2.5-Coder-32B-Instruct
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Browses the web to find information
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name                        ┃ Description                       ┃ Arguments                         ┃
            ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search                  │ Performs a google web search for  │ query (`string`): The search      │
            │                             │ your query then returns a string  │ query to perform.                 │
            │                             │ of the top search results.        │ filter_year (`integer`):          │
            │                             │                                   │ Optionally restrict results to a  │
            │                             │                                   │ certain year                      │
            │ visit_webpage               │ Visits a webpage at the given url │ url (`string`): The url of the    │
            │                             │ and reads its content as a        │ webpage to visit.                 │
            │                             │ markdown string. Use this to      │                                   │
            │                             │ browse webpages.                  │                                   │
            │ calculate_cargo_travel_time │ Calculate the travel time for a   │ origin_coords (`array`): Tuple of │
            │                             │ cargo plane between two points on │ (latitude, longitude) for the     │
            │                             │ Earth using great-circle          │ starting point                    │
            │                             │ distance.     

In [ ]:
manager_agent.run("""
Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W).
Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in total.
Represent this as spatial map of the world, with the locations represented as scatter points with a color that depends on the travel time, and save it to saved_map.png!

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,
     color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)
fig.show()
fig.write_image("saved_image.png")
final_answer(fig)

Never try to process strings using code: when you have a string to read, just print it and you'll see it.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W).                                                                             │
│ Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in     │
│ total.                                                                                                          │
│ Represent this as spatial map of the world, with the locations represented as scatter points with a color that  │
│ depends on the travel time, and save it to saved_map.png!                                                       │
│                                                                                                                 │
│ Here's an example of how to plot and return a map:                                                              │
│ import plotly.express as px                                                                                     │
│ df = px.data.carshare()                                                                                         │
│ fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,      │
│      color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)                                    │
│ fig.show()                                                                                                      │
│ fig.write_image("saved_image.png")                                                                              │
│ final_answer(fig)                                                                                               │
│                                                                                                                 │
│ Never try to process strings using code: when you have a string to read, just print it and you'll see it.       │
│                                                                                                                 │
╰─ InferenceClientModel - deepseek-ai/DeepSeek-R1 ──────────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here is the plan of action that I will follow to solve the task:
                ```
                <think>
Okay, let's break down the problem step by step. The user wants a map showing Batman filming locations and supercar
factories with their cargo plane transfer times to Gotham. The total points should be at least six, with supercar 
factories having the same transfer times as some Batman locations.

First, I need to gather the Batman filming locations. Since I don't have that data, I'll need to use the web_agent 
to search for reliable sources. Once I have the list of locations with their coordinates, I can calculate the 
travel time from each to Gotham using the calculate_cargo_travel_time tool. 

Next, I need to find supercar factories. Again, the web_agent can help find their locations. For each factory, I'll
calculate the travel time to Gotham. Then, I have to match these times with the Batman locations' times. If a 
factory's time matches any of the Batman times, it gets included. If there aren't enough matches, I might need to 
adjust by finding more factories or relaxing the time matching criteria, but the user specified "same time," so 
I'll stick to exact matches as much as possible.

Once I have at least six points (combination of Batman locations and matching factories), I'll compile them into a 
dataset. The example code uses a DataFrame with lat, lon, name, and color based on travel time. I'll structure my 
data similarly. Finally, I'll plot the points on a map using plotly, color-coding by travel time, and save the 
image.
</think>

1. Task the web agent to compile a list of all Batman movie filming locations worldwide, including their geographic
coordinates (latitude/longitude).  
2. For each Batman filming location, use the `calculate_cargo_travel_time` tool to determine the cargo plane 
transfer time to Gotham (40.7128° N, 74.0060° W).  
3. Task the web agent to identify factories of supercar manufacturers globally, including their geographic 
coordinates.  
4. For each supercar factory, calculate its cargo plane transfer time to Gotham using the same tool.  
5. Filter the supercar factories to retain only those with transfer times matching the Batman locations’ calculated
times. Combine these with Batman locations to ensure at least 6 total points.  
6. Create a DataFrame with columns for latitude, longitude, location/factory name, and transfer time.  
7. Plot the combined data on a global map using `plotly.express`, coloring points by transfer time. Save the map to
`saved_map.png`.  
8. Return the saved map as the final answer.  


                ```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
 [STEP 1]:                                                                                                         
Thought: I need to get the list of Batman filming locations with their coordinates. Since I don't have this data,  
I'll task the web_agent to search for it.                                                                          
Code:                                                                                                              
```py                                                                                                              
batman_locations_task = "Find all Batman movie filming locations worldwide. For each location, provide the name and
geographic coordinates (latitude, longitude). Use reliable sources like IMDb or Movie-Locations.com."              
web_agent_response = web_agent(task=batman_locations_task)                                                         
print(web_agent_response)                                                                                          
```<end_code>                                                                                                      
                                                                                                                   

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_locations_task = "Find all Batman movie filming locations worldwide. For each location, provide the name  
  and geographic coordinates (latitude, longitude). Use reliable sources like IMDb or Movie-Locations.com."        
  web_agent_response = web_agent(task=batman_locations_task)                                                       
  print(web_agent_response)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Reconstruct batman_times with original structure (from Step 2)                                                 
  batman_data = [                                                                                                  
      ("Stearns Wharf, Santa Barbara", 34.4188, -119.6951, 6.93),                                                  
      ("Cliffs, Marineland", 33.9, -118.4, 6.80),                                                                  
      ("Bronson Caves, Griffith Park", 34.1347, -118.3215, 6.78),                                                  
      ("Alcatraz Island", 37.8267, -122.4233, 7.05),                                                               
      ("Knebworth House, UK", 51.8055, -0.4035, 9.13),                                                             
      ("Skaftafell, Iceland", 64.2128, -19.0649, 7.37),                                                            
      ("Chicago Board of Trade", 41.8879, -87.6235, 2.68),                                                         
      ("St Pancras Hotel, London", 51.5343, -0.1341, 9.17),                                                        
      ("Battersea Power Station", 51.4719, -0.1167, 9.17),                                                         
      ("Necropolis Cemetery", 55.8642, -4.2518, 8.60)                                                              
  ]                                                                                                                
                                                                                                                   
  supercar_data = [                                                                                                
      ("Ferrari Maranello", 44.58, 10.89, 10.72),                                                                  
      ("Lamborghini Sant'Agata", 44.54, 11.18, 10.75),                                                             
      ("Bugatti Molsheim", 48.69, 7.71, 10.10)                                                                     
  ]                                                                                                                
                                                                                                                   
  # Combine into DataFrame                                                                                         
  import pandas as pd                                                                                              
  df = pd.DataFrame(                                                                                               
      [                                                                                                            
          {                                                                                                        
              'name': name,                                                                                        
              'lat': lat,                                                                                          
              'lon': lon,                                                                                          
              'time': time,                                                                                        
              'type': 'Batman Location'                                                                            
          } for (name, lat, lon, time) in batman_data                                                              
      ][38;2;248;248;

Out - Final answer: saved_map.png

Feedback:  **Reasons Why Yes:**

1. **Plotting Method**: The plot was made using `px.scatter_mapbox`, which is the correct method as per the task requirements.
2. **Location Representation**: The map shows both Batman filming locations and supercar factories, fulfilling the request for a spatial representation.
3. **Color Coding**: Locations are color-coded based on travel time, providing a clear visual differentiation.
4. **Number of Points**: The plot displays at least six points, meeting the minimum requirement set by the task.

**Reasons Why No:**

1. **Matching Times**: The task required supercar factories to match Batman locations' transfer times. This plot doesn't specifically confirm any matching times.
2. **Transfer Time Calculation**: There was a logical error in handling the transfer times that wasn't corrected in plotting. 
3. **Variable Errors**: The plot creation initially faced issues due to handling errors in data structures, affecting data integrity.

**Final Decision:

[Step 6: Duration 37.00 seconds| Input tokens: 34,990 | Output tokens: 2,560]

'saved_map.png'

I don't know how that went in your run, but in mine, the manager agent skilfully divided tasks given to the web agent in `1. Search for Batman filming locations`, then `2. Find supercar factories`, before aggregating the lists and plotting the map.

Let's see what the map looks like by inspecting it directly from the agent state:

In [ ]:
manager_agent.python_executor.state["fig"]

![output map](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/unit2/smolagents/output_map.png)